# VSB Matching

**Summary:**
In this notebook I address the first part of question 2. I use the algorithm given in the question, but I make some small modifications. I get rid of the duplicates in the dataset and I change the way that schools rank students.

In [ ]:
import pandas as pd
import numpy as np
import random

Ignore this if you are running this on Jupyter Notebook

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


## Data Cleaning

### Data Loading

In [ ]:
vsb = pd.read_csv('./anonymized.csv')
vsb

,Unnamed: 0,First Name,CHOICE,ProgramName,sIndex,open,matched,augmented
0,1894.0,Grayson,1,Montessorri Alternate Program - Renfrew,277479538,True,False,1.668609664437562
1,1505.0,Jeved,1,Early Mandarin Bilingual - Norquay,277731278,True,False,0.7405798479018244
2,669.0,Jeved,3,Early French Immersion - Laura Secord,277731278,True,False,0.7405798479018244
3,1044.0,Jeved,2,Early French Immersion - Selkirk,277731278,True,False,0.7405798479018244
4,349.0,Phoenix,2,Early French Immersion - Jules Quesnel/Queen E...,277792112,True,False,0.6796190752791323
...,...,...,...,...,...,...,...,...
4228,1802.0,Freya,1,Montessorri Alternate Program - Maple Grove,293767942,True,False,0.006012682146833925
4229,415.0,Freya,3,Early French Immersion - Jules Quesnel/Queen E...,293767942,True,False,0.006012682146833925
4230,323.0,Roselynn,1,Early French Immersion - Jules Quesnel/Queen E...,293785580,True,False,1.0261935749345041
4231,1493.0,Roselynn,3,Early French Immersion - Trafalgar,293785580,True,False,0.026193574934504027


In [ ]:
vsb = vsb.sort_values(by=['CHOICE','ProgramName','augmented'],ascending=[True,True,False])
vsb

,Unnamed: 0,First Name,CHOICE,ProgramName,sIndex,open,matched,augmented
1445,24.0,Logan,1,Early French Immersion - Douglas Annex,293447062,True,False,1.9658771610260044
3562,24.0,Logan,1,Early French Immersion - Douglas Annex,293447062,True,False,1.9658771610260044
58,13.0,Havana,1,Early French Immersion - Douglas Annex,279131054,True,False,1.9046112008246685
2175,13.0,Havana,1,Early French Immersion - Douglas Annex,279131054,True,False,1.9046112008246685
1811,10.0,Danica,1,Early French Immersion - Douglas Annex,293466534,True,False,1.7432710245847054
...,...,...,...,...,...,...,...,...
1405,2106.0,Erica,3,Montessorri Alternate Program - Tyee,293445546,True,False,0.0424629050291071
3522,2106.0,Erica,3,Montessorri Alternate Program - Tyee,293445546,True,False,0.0424629050291071
1466,2105.0,Everly,3,Montessorri Alternate Program - Tyee,293447692,True,False,0.021763956400514872
3583,2105.0,Everly,3,Montessorri Alternate Program - Tyee,293447692,True,False,0.021763956400514872


### Drop duplicates

Looking at the dataset, it appears that the data is has a plethora of data duplicates. Removing these duplicates reduced the dataset's size by half

In [ ]:
vsb = vsb.drop_duplicates()

In [ ]:
vsb

,Unnamed: 0,First Name,CHOICE,ProgramName,sIndex,open,matched,augmented
1445,24.0,Logan,1,Early French Immersion - Douglas Annex,293447062,True,False,1.9658771610260044
58,13.0,Havana,1,Early French Immersion - Douglas Annex,279131054,True,False,1.9046112008246685
1811,10.0,Danica,1,Early French Immersion - Douglas Annex,293466534,True,False,1.7432710245847054
280,35.0,Ronan,1,Early French Immersion - Douglas Annex,281738796,True,False,1.6983503260550585
354,3.0,Brett,1,Early French Immersion - Douglas Annex,281991244,True,False,1.5152183713808318
...,...,...,...,...,...,...,...,...
1855,2113.0,Laurence,3,Montessorri Alternate Program - Tyee,293468348,True,False,0.1309436006959116
299,2099.0,Soelle,3,Montessorri Alternate Program - Tyee,281786364,True,False,0.09480665187918513
1405,2106.0,Erica,3,Montessorri Alternate Program - Tyee,293445546,True,False,0.0424629050291071
1466,2105.0,Everly,3,Montessorri Alternate Program - Tyee,293447692,True,False,0.021763956400514872


## Assigning  a Global Ranking to Each Student

### Using randomized uniform floats from 0 to 1 to rank each student

In [ ]:
students = vsb.drop_duplicates(subset=['sIndex'])
students = students[['sIndex']]
students['Ranking'] = np.random.uniform(0, 1, students.shape[0])

students


,sIndex,Ranking
1445,293447062,0.996681
58,279131054,0.027968
1811,293466534,0.676703
280,281738796,0.302827
354,281991244,0.461485
...,...,...
1849,293468248,0.243297
810,293429540,0.990497
549,285046234,0.098907
884,293430976,0.632219


### Attaching the rankings to the dataset and dropping an extra column to clean the data further

In [ ]:
newVsb = pd.merge(students,vsb,on='sIndex')
newVsb = newVsb.drop(columns=['augmented'])
newVsb = newVsb.drop(2116)
newVsb

,sIndex,Ranking,Unnamed: 0,First Name,CHOICE,ProgramName,open,matched
0,293447062,0.996681,24.0,Logan,1,Early French Immersion - Douglas Annex,True,False
1,293447062,0.996681,1598.0,Logan,2,Early Mandarin Bilingual - Norquay,True,False
2,293447062,0.996681,1941.0,Logan,3,Montessorri Alternate Program - Renfrew,True,False
3,279131054,0.027968,13.0,Havana,1,Early French Immersion - Douglas Annex,True,False
4,293466534,0.676703,10.0,Danica,1,Early French Immersion - Douglas Annex,True,False
...,...,...,...,...,...,...,...,...
2111,293429540,0.990497,1042.0,Mark,2,Early French Immersion - Selkirk,True,False
2112,293429540,0.990497,1730.0,Mark,3,Fine Arts - Nootka,True,False
2113,285046234,0.098907,1973.0,Lila,1,Montessorri Alternate Program - Tyee,True,False
2114,285046234,0.098907,1047.0,Lila,2,Early French Immersion - Selkirk,True,False


### Importing the .json data file

In [ ]:
import json
with open("./schools_data.json", "r") as read_file:
    schools = json.load(read_file)

schools

{'Early French Immersion - Douglas Annex': {'members': [], 'seats': 60},
 'Early French Immersion - Hastings': {'members': [], 'seats': 40},
 'Early French Immersion - Hudson': {'members': [], 'seats': 20},
 'Early French Immersion - Jules Quesnel/Queen Elizabeth Annex': {'members': [],
  'seats': 40},
 'Early French Immersion - Kerrisdale': {'members': [], 'seats': 40},
 "Early French Immersion - L'Ecole Bilingue": {'members': [], 'seats': 60},
 'Early French Immersion - Laura Secord': {'members': [], 'seats': 40},
 'Early French Immersion - Quilchena': {'members': [], 'seats': 20},
 'Early French Immersion - Selkirk': {'members': [], 'seats': 40},
 'Early French Immersion - Strathcona': {'members': [], 'seats': 20},
 'Early French Immersion - Tennyson': {'members': [], 'seats': 60},
 'Early French Immersion - Trafalgar': {'members': [], 'seats': 40},
 'Early Mandarin Bilingual - Norquay': {'members': [], 'seats': 20},
 'Fine Arts - Nootka': {'members': [], 'seats': 20},
 "Indigenous 

Here I found that there is a sum of 600 seats in all the schools combined.

In [ ]:
sum = 0
for school in schools:
    sum += schools[school]['seats']
    
sum

600

## Deffered Acceptance Algorithm

I modified this data to take in my new data variable and fixed some minor mistakes in the starter file provided to us. The error spotted was that the algorithm was changing strings of "False" or "True", into boolean variables. Consequently, it lead to the wrong output.

In [ ]:
# sIndex
def proposal(sIndex,Ranking,ProgramName,newVsb,schools):
    """
    A student represented by sIndex proposes to a school called ProgramName.
    sIndex - a numeric id number for a student, links to a specific row in the ds dataframe
    augmented - the students rank used to decide whether to displace existing applicants
    ProgramName - the text string associated with a school
    ds - the dataframe associated with the students preferences
    schools - a dictionary of school names mapped to arrays with student index numbers(sIndex)
    return - True if the schools is currently willing to accept a student, False otherwise.
    ****
    The routine checks if the school currently has space, if so it appends the student id to its
    member list, changes the matching field to True in the ds data frame and returns true.  
    If not, it checks to see whether the applicant is better than the worst student
    in its list.  If so, it removes the worst student from its member list, changes the open
    field in the data frame ds for the student to False, adds the new applicant to its members
    list, and changes the matching field for the new applicant to true, then returns true.
    Otherwise it switches the open fields in the data frame to false and returns false.
    """ 
    # first a sanity check - is it there already
    list = {}
    if sIndex in schools[ProgramName]['members']:
      
        return False
        #print("already there")
           
    
    # is there room? if yes add it, otherwise go on
    if len(schools[ProgramName]['members']) < schools[ProgramName]['seats']:
       # print(schools[ProgramName]['seats'])
        #return
        schools[ProgramName]['members'].append(sIndex)
        newVsb.loc[(newVsb['ProgramName'] == ProgramName)&(newVsb['sIndex'] == sIndex),['matched']]='True'
        return True
    #create a list of current members with rankings
    for spin in schools[ProgramName]['members'] :
        f = newVsb.loc[(newVsb['ProgramName'] == ProgramName)&(newVsb['sIndex'] == spin)]
        list[spin] =f.iloc[0]['Ranking'] 
        del f
    #get the minimum value in the new list
    v = min(list,key=list.get)
    if Ranking <= list[v]:
        #reject
        newVsb.loc[(newVsb['ProgramName'] == ProgramName)&(newVsb['sIndex'] == sIndex),['open']]='False'
        return False
    #remove the old one
    newVsb.loc[(newVsb['ProgramName'] == ProgramName)&(newVsb['sIndex'] == v),['open']]='False'
    newVsb.loc[(newVsb['ProgramName'] == ProgramName)&(newVsb['sIndex'] == v),['matched']]='False'
    schools[ProgramName]['members'].remove(v)
    # add the new one
    schools[ProgramName]['members'].append(sIndex)
    newVsb.loc[(newVsb['ProgramName'] == ProgramName)&(newVsb['sIndex'] == sIndex),['matched']]='True'
    return True
    
def mainAlgo():    
  current = 0
  for index,element in newVsb.iterrows():
      if element['matched'] == 'False' and current != element['sIndex'] and element['open'] == 'True':
          test = proposal(element['sIndex'],element['Ranking'],element['ProgramName'],newVsb,schools)
          current = element['sIndex']  
      elif element['matched'] == 'True':
          current = element['sIndex']
      else:
          continue

mainAlgo() # run 3 times to ensure all students are assigned 
mainAlgo()
mainAlgo()

dss = newVsb[newVsb['matched'] == 'True']
dss

,sIndex,Ranking,Unnamed: 0,First Name,CHOICE,ProgramName,open,matched
0,293447062,0.996681,24.0,Logan,1,Early French Immersion - Douglas Annex,True,True
4,293466534,0.676703,10.0,Danica,1,Early French Immersion - Douglas Annex,True,True
5,281738796,0.302827,35.0,Ronan,1,Early French Immersion - Douglas Annex,True,True
8,281991244,0.461485,3.0,Brett,1,Early French Immersion - Douglas Annex,True,True
9,281882706,0.250220,14.0,Alvin,1,Early French Immersion - Douglas Annex,True,True
...,...,...,...,...,...,...,...,...
2092,285678730,0.578490,913.0,Tara,3,Early French Immersion - L'Ecole Bilingue,True,True
2099,279818920,0.502761,1733.0,Hunter,3,Fine Arts - Nootka,True,True
2103,283367908,0.968361,1998.0,Danay,1,Montessorri Alternate Program - Tyee,True,True
2109,293468248,0.243297,82.0,Paulina,3,Early French Immersion - Douglas Annex,True,True


## Final Results

In [ ]:
newVsb

,sIndex,Ranking,Unnamed: 0,First Name,CHOICE,ProgramName,open,matched
0,293447062,0.996681,24.0,Logan,1,Early French Immersion - Douglas Annex,True,True
1,293447062,0.996681,1598.0,Logan,2,Early Mandarin Bilingual - Norquay,True,False
2,293447062,0.996681,1941.0,Logan,3,Montessorri Alternate Program - Renfrew,True,False
3,279131054,0.027968,13.0,Havana,1,Early French Immersion - Douglas Annex,False,False
4,293466534,0.676703,10.0,Danica,1,Early French Immersion - Douglas Annex,True,True
...,...,...,...,...,...,...,...,...
2111,293429540,0.990497,1042.0,Mark,2,Early French Immersion - Selkirk,True,False
2112,293429540,0.990497,1730.0,Mark,3,Fine Arts - Nootka,True,False
2113,285046234,0.098907,1973.0,Lila,1,Montessorri Alternate Program - Tyee,False,False
2114,285046234,0.098907,1047.0,Lila,2,Early French Immersion - Selkirk,False,False


In [ ]:
dss = newVsb[newVsb['matched'] == True]
dss

,sIndex,Ranking,Unnamed: 0,First Name,CHOICE,ProgramName,open,matched


In [ ]:
for school in schools:
    print(school,schools[school]['seats'],len(schools[school]['members']))

Early French Immersion - Selkirk 40 40
Early French Immersion - Tennyson 60 60
Early French Immersion - Hastings 40 40
Fine Arts - Nootka 20 20
Early French Immersion - Strathcona 20 20
Montessorri Alternate Program - Maple Grove 20 20
Early Mandarin Bilingual - Norquay 20 20
Montessorri Alternate Program - Renfrew 20 20
Early French Immersion - Laura Secord 40 40
Early French Immersion - Jules Quesnel/Queen Elizabeth Annex 40 40
Early French Immersion - Kerrisdale 40 40
Indigenous Focus - Xpey' 20 2
Early French Immersion - Trafalgar 40 40
Early French Immersion - Quilchena 20 20
Montessorri Alternate Program - Tyee 20 20
Early French Immersion - Hudson 20 20
Early French Immersion - L'Ecole Bilingue 60 60
Early French Immersion - Douglas Annex 60 60


In [ ]:
dst = newVsb[newVsb.matched == 'True']
dst['CHOICE'].value_counts()

1    483
2     73
3     26
Name: CHOICE, dtype: int64